In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.interpolate import interp1d
from tqdm import tqdm
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore')

In [66]:
import geopandas as gpd
import pandas as pd
import numpy as np
from tqdm import tqdm

def sliding_window_segmentation(data, window_size, step_size, mmsi):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        window = data.iloc[start:start + window_size].copy()
        window['MMSI'] = mmsi
        segments.append(window)
    return segments

def lat_lon_range(df_file):
    min_lat, max_lat = df_file['LAT'].min(), df_file['LAT'].max()
    min_lon, max_lon = df_file['LON'].min(), df_file['LON'].max()
    min_lat, max_lat, min_lon, max_lon = (
        int(np.floor(min_lat)),
        int(np.ceil(max_lat)),
        int(np.floor(min_lon)),
        int(np.ceil(max_lon)),
    )
    return min_lat, max_lat, min_lon, max_lon

def prepare_ais_data_without_interpolation(
    data, bounding_box, window_size, step_size
):
    min_lat, max_lat, min_lon, max_lon = bounding_box

    filtered_data = data[
        (data['LAT'] >= min_lat) & (data['LAT'] <= max_lat) &
        (data['LON'] >= min_lon) & (data['LON'] <= max_lon)
    ]

    grouped = filtered_data.groupby('MMSI')
    segments = []
    segment_id = 0

    for mmsi, group in tqdm(grouped, desc="Processing MMSI", unit="MMSI"):
        group = group.sort_values(by='BaseDateTime')
        group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])

        # Directly use the existing data without interpolation
        resampled_traj = group[['BaseDateTime', 'LAT', 'LON', 'SOG', 'COG']].copy()

        trajectory_segments = sliding_window_segmentation(resampled_traj, window_size, step_size, mmsi)

        for segment in trajectory_segments:
            segment['SegmentID'] = segment_id
            segments.append(segment)
            segment_id += 1

    combined_df = pd.concat(segments, ignore_index=True)
    return combined_df

# Example usage
if __name__ == "__main__":
    df = pd.read_csv("/content/modified_ais_data.csv")[:100000]

    # Define bounding box from the dataset
    min_lat, max_lat, min_lon, max_lon = lat_lon_range(df)
    bounding_box = (min_lat, max_lat, min_lon, max_lon)

    window_size = 10
    step_size = 5

    segmented_trajectories_df = prepare_ais_data_without_interpolation(
        df, bounding_box, window_size, step_size
    )

    segmented_trajectories_df.to_csv("segmented_trajectories.csv", index=False)


Processing MMSI: 100%|██████████| 3093/3093 [00:20<00:00, 150.31MMSI/s]


### calculate duration

In [67]:
def calculate_durations(group):
    # Ensure BaseDateTime is a datetime type
    group['BaseDateTime'] = pd.to_datetime(group['BaseDateTime'])
    # Calculate the duration between consecutive points in seconds
    group['duration_seconds'] = group['BaseDateTime'].diff().dt.total_seconds()
    return group

In [68]:
segmented_trajectories_df = segmented_trajectories_df.sort_values(by=['BaseDateTime'])
segmented_trajectories_df = segmented_trajectories_df.groupby("MMSI", group_keys=False).apply(calculate_durations)
segmented_trajectories_df = segmented_trajectories_df.reset_index(drop=True)
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID,duration_seconds
0,2022-03-31,32.65303,-117.12239,1.7,166.4,366764730,4891,NaN
1,2022-03-31,34.36573,-121.77418,7.9,278.0,356909000,4471,NaN
2,2022-03-31,48.47441,-125.41904,1.0,115.0,357175000,4478,NaN
3,2022-03-31,29.74757,-95.11144,0.0,238.0,367141680,6735,NaN
4,2022-03-31,24.08663,-82.27491,9.4,57.0,319874000,3544,NaN


In [69]:
segmented_trajectories_df.isna().sum()

,0
BaseDateTime,0
LAT,0
LON,0
SOG,0
COG,0
MMSI,0
SegmentID,0
duration_seconds,2750


In [70]:
segmented_trajectories_df.shape

(158270, 8)

In [71]:
segmented_trajectories_df.fillna(0,inplace=True)

In [72]:
segmented_trajectories_df.isna().sum()

,0
BaseDateTime,0
LAT,0
LON,0
SOG,0
COG,0
MMSI,0
SegmentID,0
duration_seconds,0


### calculate x,y,z

In [73]:
import math

def lat_lon_to_cartesian(lat, lon, R=6371):
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    x = R * math.cos(lat_rad) * math.cos(lon_rad)
    y = R * math.cos(lat_rad) * math.sin(lon_rad)
    z = R * math.sin(lat_rad)
    return x, y, z

def cartesian_to_lat_lon(x, y, z, R=6371):
    lon = math.degrees(math.atan2(y, x))
    lat = math.degrees(math.asin(z / R))
    return lat, lon


In [74]:
segmented_trajectories_df[['x', 'y', 'z']] = segmented_trajectories_df.apply(lambda row: pd.Series(lat_lon_to_cartesian(row['LAT'], row['LON'])), axis=1)
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID,duration_seconds,x,y,z
0,2022-03-31,32.65303,-117.12239,1.7,166.4,366764730,4891,0.0,-2445.447444,-4774.221861,3437.474862
1,2022-03-31,34.36573,-121.77418,7.9,278.0,356909000,4471,0.0,-2769.219752,-4470.791605,3596.259916
2,2022-03-31,48.47441,-125.41904,1.0,115.0,357175000,4478,0.0,-2447.844023,-3442.028194,4769.710950
3,2022-03-31,29.74757,-95.11144,0.0,238.0,367141680,6735,0.0,-492.812521,-5509.431869,3161.160752
4,2022-03-31,24.08663,-82.27491,9.4,57.0,319874000,3544,0.0,781.824216,-5763.487453,2600.116203


### calculate distance

In [75]:
import numpy as np
def haversine(lat1, lon1, lat2, lon2, R=6371):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def calculate_distances(group):
    group = group.sort_values(by="BaseDateTime")
    group["next_lat"] = group["LAT"].shift(-1)
    group["next_long"] = group["LON"].shift(-1)
    group["distance_km"] = group.apply(
        lambda row: haversine(row["LAT"], row["LON"], row["next_lat"], row["next_long"])
        if not np.isnan(row["next_lat"])
        else 0,
        axis=1,
    )
    return group.drop(columns=["next_lat", "next_long"])

In [76]:
segmented_trajectories_df = segmented_trajectories_df.groupby("MMSI").apply(calculate_distances).reset_index(drop=True)
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID,duration_seconds,x,y,z,distance_km
0,2022-03-31 00:02:32,27.35372,-94.62546,0.4,228.6,111,0,0.0,-456.323030,-5640.208831,2927.363085,0.003950
1,2022-03-31 00:05:35,27.35372,-94.62550,0.6,219.8,111,0,183.0,-456.326968,-5640.208513,2927.363085,0.008126
2,2022-03-31 00:08:34,27.35377,-94.62556,0.2,221.7,111,0,179.0,-456.332668,-5640.205489,2927.368023,0.003479
3,2022-03-31 00:11:31,27.35380,-94.62557,0.3,105.0,111,0,177.0,-456.333529,-5640.203881,2927.370986,0.022308
4,2022-03-31 00:14:33,27.35365,-94.62542,0.3,173.4,111,0,182.0,-456.319381,-5640.212715,2927.356172,0.002224


### Convert SOG to meters per second

In [77]:
segmented_trajectories_df['sog_to_meters_per_second'] =  segmented_trajectories_df.apply(lambda row: row['SOG'] * 0.514444, axis=1)
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID,duration_seconds,x,y,z,distance_km,sog_to_meters_per_second
0,2022-03-31 00:02:32,27.35372,-94.62546,0.4,228.6,111,0,0.0,-456.323030,-5640.208831,2927.363085,0.003950,0.205778
1,2022-03-31 00:05:35,27.35372,-94.62550,0.6,219.8,111,0,183.0,-456.326968,-5640.208513,2927.363085,0.008126,0.308666
2,2022-03-31 00:08:34,27.35377,-94.62556,0.2,221.7,111,0,179.0,-456.332668,-5640.205489,2927.368023,0.003479,0.102889
3,2022-03-31 00:11:31,27.35380,-94.62557,0.3,105.0,111,0,177.0,-456.333529,-5640.203881,2927.370986,0.022308,0.154333
4,2022-03-31 00:14:33,27.35365,-94.62542,0.3,173.4,111,0,182.0,-456.319381,-5640.212715,2927.356172,0.002224,0.154333


### Convert COG to radians

In [78]:
segmented_trajectories_df['cog_to_radians'] = segmented_trajectories_df.apply(lambda row: np.radians(row['COG']) , axis=1)
segmented_trajectories_df.head()

,BaseDateTime,LAT,LON,SOG,COG,MMSI,SegmentID,duration_seconds,x,y,z,distance_km,sog_to_meters_per_second,cog_to_radians
0,2022-03-31 00:02:32,27.35372,-94.62546,0.4,228.6,111,0,0.0,-456.323030,-5640.208831,2927.363085,0.003950,0.205778,3.989823
1,2022-03-31 00:05:35,27.35372,-94.62550,0.6,219.8,111,0,183.0,-456.326968,-5640.208513,2927.363085,0.008126,0.308666,3.836234
2,2022-03-31 00:08:34,27.35377,-94.62556,0.2,221.7,111,0,179.0,-456.332668,-5640.205489,2927.368023,0.003479,0.102889,3.869395
3,2022-03-31 00:11:31,27.35380,-94.62557,0.3,105.0,111,0,177.0,-456.333529,-5640.203881,2927.370986,0.022308,0.154333,1.832596
4,2022-03-31 00:14:33,27.35365,-94.62542,0.3,173.4,111,0,182.0,-456.319381,-5640.212715,2927.356172,0.002224,0.154333,3.026401


In [79]:
segmented_trajectories_df = segmented_trajectories_df[['x','y','z','duration_seconds', 'distance_km', 'sog_to_meters_per_second', 'cog_to_radians','SegmentID']]
segmented_trajectories_df.head()

,x,y,z,duration_seconds,distance_km,sog_to_meters_per_second,cog_to_radians,SegmentID
0,-456.323030,-5640.208831,2927.363085,0.0,0.003950,0.205778,3.989823,0
1,-456.326968,-5640.208513,2927.363085,183.0,0.008126,0.308666,3.836234,0
2,-456.332668,-5640.205489,2927.368023,179.0,0.003479,0.102889,3.869395,0
3,-456.333529,-5640.203881,2927.370986,177.0,0.022308,0.154333,1.832596,0
4,-456.319381,-5640.212715,2927.356172,182.0,0.002224,0.154333,3.026401,0


In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm


# Filter segments with insufficient rows
def filter_short_segments(data, window_size, prediction_horizon, group_col='SegmentID'):
    min_required_rows = window_size + prediction_horizon
    filtered_data = data.groupby(group_col).filter(lambda x: len(x) >= min_required_rows)
    print(f"Filtered data contains {len(filtered_data)} rows after removing short segments.")
    return filtered_data


# Prepare sequences with temporal features and SegmentID
def prepare_sequences(data, window_size, prediction_horizon,
                      group_col='SegmentID', time_col='BaseDateTime'):
    sequences = []
    unique_segments = sorted(data[group_col].unique())

    for segment in tqdm(unique_segments, desc="Processing Segments", unit="segment"):
        segment_data = data[data[group_col] == segment].reset_index(drop=True)
        num_rows = len(segment_data)
        # print(f"Segment {segment} has {num_rows} rows.")

        if num_rows < window_size + prediction_horizon:
            # print(f"Skipping segment {segment}: Not enough rows.")
            continue

        for start in range(num_rows - window_size - prediction_horizon + 1):
            x_window = segment_data.iloc[start:start+window_size]
            y_window = segment_data.iloc[start+window_size:start+window_size+prediction_horizon]
            sequences.append({
                'X': {
                    'x': x_window['x'].tolist(),
                    'y': x_window['y'].tolist(),
                    'z': x_window['z'].tolist(),
                    'duration_seconds': x_window['duration_seconds'].tolist(),
                    'distance_km': x_window['distance_km'].tolist(),
                    'sog_to_meters_per_second': x_window['sog_to_meters_per_second'].tolist(),
                    'cog_to_radians': x_window['cog_to_radians'].tolist(),
                    'SegmentID': x_window['SegmentID'].tolist(),
                },
                'Y': {
                   'x': x_window['x'].tolist(),
                    'y': x_window['y'].tolist(),
                    'z': x_window['z'].tolist(),
                    'duration_seconds': x_window['duration_seconds'].tolist(),
                    'distance_km': x_window['distance_km'].tolist(),
                    'sog_to_meters_per_second': x_window['sog_to_meters_per_second'].tolist(),
                    'cog_to_radians': x_window['cog_to_radians'].tolist(),
                    'SegmentID': x_window['SegmentID'].tolist(),
                }
            })
    return sequences


# Split data into train/val/test
def split_train_val_test(data, test_size=0.2, val_size=0.2):
    train_data, temp_data = train_test_split(data, test_size=test_size + val_size, random_state=42, shuffle=True)
    val_data, test_data = train_test_split(temp_data, test_size=test_size / (test_size + val_size), random_state=42, shuffle=True)
    return train_data, val_data, test_data


# Data preparation pipeline without scaling
def prepare_data_pipeline_no_scaling(data, window_size, prediction_horizon,
                                     test_size=0.2, val_size=0.2):
    # Filter short segments
    print("Filtering short segments...")
    data = filter_short_segments(data, window_size, prediction_horizon)

    # Split data
    print("Splitting data into train, validation, and test sets...")
    train_data, val_data, test_data = split_train_val_test(data, test_size, val_size)

    # Prepare sequences
    print("Preparing training sequences...")
    train_sequences = prepare_sequences(train_data, window_size, prediction_horizon)

    print("Preparing validation sequences...")
    val_sequences = prepare_sequences(val_data, window_size, prediction_horizon)

    print("Preparing test sequences...")
    test_sequences = prepare_sequences(test_data, window_size, prediction_horizon)

    return train_sequences, val_sequences, test_sequences

# Parameters
window_size = 5
prediction_horizon = 2
test_size = 0.2
val_size = 0.32


# Prepare data
train_sequences, val_sequences, test_sequences = prepare_data_pipeline_no_scaling(
    segmented_trajectories_df, window_size, prediction_horizon, test_size, val_size
)

print(f"Number of training sequences: {len(train_sequences)}")
print(f"Number of validation sequences: {len(val_sequences)}")
print(f"Number of test sequences: {len(test_sequences)}")



Filtering short segments...
Filtered data contains 158270 rows after removing short segments.
Splitting data into train, validation, and test sets...
Preparing training sequences...


Processing Segments: 100%|██████████| 15793/15793 [00:12<00:00, 1259.92segment/s]


Preparing validation sequences...


Processing Segments: 100%|██████████| 15486/15486 [00:10<00:00, 1510.86segment/s]


Preparing test sequences...


Processing Segments: 100%|██████████| 14065/14065 [00:10<00:00, 1334.58segment/s]

Number of training sequences: 3084
Number of validation sequences: 287
Number of test sequences: 14


In [85]:
import numpy as np

def format_sequences(sequences, input_features, output_features, predicted_seq_len):
    """
    Format sequences for Seq2Seq model.

    Parameters:
    - sequences: List of dictionaries with 'X' and 'Y'.
    - input_features: List of feature names for inputs.
    - output_features: List of feature names for outputs.
    - predicted_seq_len: Length of the predicted sequence.

    Returns:
    - encoder_inputs: NumPy array of shape (num_sequences, input_seq_len, num_features).
    - decoder_inputs: NumPy array of shape (num_sequences, output_seq_len, num_features).
    - targets: NumPy array of shape (num_sequences, output_seq_len, num_targets).
    """
    encoder_inputs = []
    decoder_inputs = []
    targets = []

    for seq in sequences:
        # Extract input features for encoder
        encoder_input = np.array([seq['X'][feature] for feature in input_features]).T
        encoder_inputs.append(encoder_input)

        # Extract Y data for decoder inputs and targets
        y_data = np.array([seq['Y'][feature] for feature in input_features]).T  # Use input features for decoder inputs
        target_data = np.array([seq['Y'][feature] for feature in output_features]).T  # Use output features for targets

        # Ensure y_data has at least predicted_seq_len timesteps
        y_data = y_data[:predicted_seq_len, :] if len(y_data) >= predicted_seq_len else np.pad(
            y_data, ((0, predicted_seq_len - len(y_data)), (0, 0)), mode='constant'
        )

        target_data = target_data[:predicted_seq_len, :] if len(target_data) >= predicted_seq_len else np.pad(
            target_data, ((0, predicted_seq_len - len(target_data)), (0, 0)), mode='constant'
        )

        # Initialize decoder input and fill with previous steps
        decoder_input = np.zeros((predicted_seq_len, len(input_features)))
        decoder_input[:min(len(y_data) - 1, predicted_seq_len - 1), :] = y_data[:min(len(y_data) - 1, predicted_seq_len - 1), :]

        decoder_inputs.append(decoder_input)
        targets.append(target_data)

    return (
        np.array(encoder_inputs),
        np.array(decoder_inputs),
        np.array(targets)
    )


# Define the features for input and output
input_features = ['x', 'y', 'z', 'duration_seconds', 'distance_km',
                  'sog_to_meters_per_second', 'cog_to_radians']

output_features = ['x', 'y', 'z', 'duration_seconds',
                  'sog_to_meters_per_second', 'cog_to_radians']

predicted_seq_len = 2

# Format sequences for training, validation, and testing
train_enc_inputs, train_dec_inputs, train_targets = format_sequences(train_sequences, input_features, output_features, predicted_seq_len)
val_enc_inputs, val_dec_inputs, val_targets = format_sequences(val_sequences, input_features, output_features, predicted_seq_len)
test_enc_inputs, test_dec_inputs, test_targets = format_sequences(test_sequences, input_features, output_features, predicted_seq_len)

# Print the shapes to verify
print("Training data shapes:")
print(f"Encoder inputs: {train_enc_inputs.shape}, Decoder inputs: {train_dec_inputs.shape}, Targets: {train_targets.shape}")

# Check for NaN or Inf in your data
print("Checking training data for NaN or Inf...")
print(np.isnan(train_enc_inputs).any(), np.isinf(train_enc_inputs).any())
print(np.isnan(train_dec_inputs).any(), np.isinf(train_dec_inputs).any())
print(np.isnan(train_targets).any(), np.isinf(train_targets).any())


Training data shapes:
Encoder inputs: (3084, 5, 7), Decoder inputs: (3084, 2, 7), Targets: (3084, 2, 6)
Checking training data for NaN or Inf...
False False
False False
False False


In [86]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Concatenate, Dropout, MultiHeadAttention,
    Add, LayerNormalization, RepeatVector, Lambda, Conv1D, Bidirectional
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, input_dim, output_dim, latent_dim):
    """
    Enhanced Probabilistic Seq2Seq Model for Time Series Forecasting with improvements for handling time series data.
    """
    # Encoder
    encoder_inputs = Input(shape=(input_seq_len, input_dim), name="encoder_inputs")

    # Temporal Convolutional Layers for better feature extraction in time series data
    encoder_conv = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="encoder_conv")(encoder_inputs)
    encoder_conv = Dropout(0.3, name="encoder_conv_dropout")(encoder_conv)

    # Bidirectional LSTM for Encoder to capture patterns from both directions
    encoder_lstm = Bidirectional(LSTM(latent_dim, return_sequences=True, return_state=True, name="encoder_lstm"))
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_conv)  # Adjusted unpacking

    # Combine forward and backward states to form initial state for decoder
    state_h = Concatenate()([forward_h, backward_h])
    state_c = Concatenate()([forward_c, backward_c])

    # Multi-Head Attention for Encoder
    multi_head_attention = MultiHeadAttention(num_heads=4, key_dim=latent_dim, name="multi_head_attention")
    encoder_attention = multi_head_attention(encoder_outputs, encoder_outputs)  # Query and value are the same
    encoder_attention = LayerNormalization(name="encoder_attention_norm")(encoder_attention)

    # Decoder
    decoder_inputs = Input(shape=(output_seq_len, input_dim), name="decoder_inputs")

    # Convolutional Layer in Decoder
    decoder_conv = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', name="decoder_conv")(decoder_inputs)
    decoder_conv = Dropout(0.3, name="decoder_conv_dropout")(decoder_conv)

    # Decoder LSTM, using encoder states for initialization
    decoder_lstm = LSTM(latent_dim * 2, return_sequences=True, return_state=True, name="decoder_lstm")
    decoder_outputs, _, _ = decoder_lstm(decoder_conv, initial_state=[state_h, state_c])

    # Ensure that the attention mechanism has compatible shapes (query and value)
    attention_context = multi_head_attention(decoder_outputs, encoder_attention)  # Using encoder attention in decoder
    attention_context = LayerNormalization(name="decoder_attention_norm")(attention_context)
    attention_context = Lambda(lambda x: tf.reduce_mean(x, axis=1), name="summarize_attention")(attention_context)

    # Ensure Attention Context and Decoder Outputs have compatible dimensions
    attention_context_repeated = RepeatVector(output_seq_len)(attention_context)  # Repeat to match output_seq_len
    attention_context_repeated = tf.keras.layers.Reshape((output_seq_len, latent_dim * 2))(attention_context_repeated)

    # Merge Attention Context with Decoder Outputs
    merged_context = Concatenate(axis=-1)([decoder_outputs, attention_context_repeated])

    # Dropout and Output Layers
    dropout = Dropout(0.4, name="decoder_dropout")(merged_context)
    output_mean = Dense(output_dim, activation="linear", kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                        name="mean_output")(dropout)
    output_std = Dense(output_dim, activation="softplus", kernel_regularizer=tf.keras.regularizers.l2(1e-4),
                       name="std_output")(dropout)

    # Combine Outputs
    outputs = Concatenate(name="probabilistic_outputs")([output_mean, output_std])

    # Define the Model
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=outputs, name="enhanced_probabilistic_seq2seq_model")

    # Custom Loss Function (Negative Log-Likelihood)
    def nll_loss(y_true, y_pred):
        mean = y_pred[..., :output_dim]
        std = y_pred[..., output_dim:]
        dist = tf.compat.v1.distributions.Normal(loc=mean, scale=std)
        return -tf.reduce_mean(dist.log_prob(y_true))

    # Probabilistic Evaluation Metric: CRPS
    def crps_metric(y_true, y_pred):
        mean = y_pred[..., :output_dim]
        std = y_pred[..., output_dim:]
        return tf.reduce_mean((y_true - mean) ** 2 / (2 * std ** 2) + tf.math.log(std))

    # Adjusted R² Metric
    def adjusted_r2_score(y_true, y_pred):
        mean = y_pred[..., :output_dim]
        ss_res = tf.reduce_sum(tf.square(y_true - mean), axis=-1)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=-1, keepdims=True)), axis=-1)
        r2 = 1 - (ss_res / (ss_tot + tf.keras.backend.epsilon()))

        # Number of samples (batch size)
        n = tf.cast(tf.shape(y_true)[0], tf.float32)

        # Number of predictors (features)
        p = tf.cast(output_dim, tf.float32)

        # Adjusted R²
        adjusted_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1 + tf.keras.backend.epsilon())
        return adjusted_r2

    # Optimizer
    optimizer = Adam(learning_rate=0.01)

    # Compile the Model
    model.compile(optimizer=optimizer, loss=nll_loss, metrics=[crps_metric, adjusted_r2_score])

    return model




In [90]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Hyperparameters
input_seq_len = 5  # Input sequence length
output_seq_len = 2  # Output sequence length
latent_dim = 64  # Number of LSTM units
input_dim = len(input_features)  # Number of input features
output_dim = len(output_features)  # Number of output features

# Create the model
model = create_advanced_probabilistic_seq2seq(input_seq_len, output_seq_len, input_dim, output_dim, latent_dim)

# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=1)
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

# Train the model
print("Training the model...")
history = model.fit(
    [train_enc_inputs, train_dec_inputs],
    train_targets,
    validation_data=([val_enc_inputs, val_dec_inputs], val_targets),
    batch_size=32,
    epochs=100,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# Save the Model
model.save_weights("prob_attention_lstm.weights.h5")
print("Model saved successfully.")


Training the model...
Epoch 1/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 28s 72ms/step - adjusted_r2_score: -0.2626 - crps_metric: 7789423.5000 - loss: 7789625.5000 - val_adjusted_r2_score: -0.2605 - val_crps_metric: 14916.7324 - val_loss: 14917.3730 - learning_rate: 0.0100
Epoch 2/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 5s 56ms/step - adjusted_r2_score: -0.2586 - crps_metric: 13935.2100 - loss: 13936.4287 - val_adjusted_r2_score: -0.2565 - val_crps_metric: 9729.5205 - val_loss: 9730.2637 - learning_rate: 0.0100
Epoch 3/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - adjusted_r2_score: -0.2538 - crps_metric: 9878.3779 - loss: 9879.2852 - val_adjusted_r2_score: -0.2532 - val_crps_metric: 7622.1250 - val_loss: 7622.9268 - learning_rate: 0.0100
Epoch 4/100
97/97 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - adjusted_r2_score: -0.2512 - crps_metric: 7941.8506 - loss: 7942.8301 - val_adjusted_r2_score: -0.2500 - val_crps_metric: 6302.7095 - val_loss: 6303.5117 - learning_rate: 0.0100
Epoch 5/100
97/97 ━━━━━━━━━━━━━━━━━━━━